This script further processes the initial 1x1km suitability layers of each hydrogen production technology. The steps are as follows:
1) The 1x1km features are broken down into sizes corresponding to the reference plant footprint of each production technology.

2) A high geospatial resolution filter (30m) is applied using the NLCD and Farms Under Threat datasets.

3) Candidates that do not fall squarely within a load zone are removed.

4) ** Requires manual intervention ** Export the intermediate outputs from Step 3 to pgAdmin and use Postgres to add distances to feedstock sources and substations using the script in the 'postgres' folder. Put outputs in the folder 'candidate_sites_with_dists'

5) Remove candidate sites that overlap with substations or above-ground feedstock sources (an overlap with a natural gas pipelines is allowed because these are typically located underground, but an overlap with a biogas plant is not)

6) Overlaps between technologies are resolved by grouping the candidate layers and iteratively filtering them in order of increasing candidate count, so that sites from sparser layers are preserved at a higher proportion.

Set-Up:

In [19]:
# Import libraries and methods

import geopandas as gpd
from pathlib import Path
from shapely.geometry import box
import rasterio
import numpy as np
import pandas as pd
from rasterio.features import geometry_mask
from rasterio.windows import from_bounds

In [8]:
# Load data paths
base_path = Path.cwd() # this is the pre-processing folder path

# Input data paths
candidate_sites_path = base_path / 'input_files' / "candidate_sites_1x1km"
combined_exclusion_30m_path = base_path / 'input_files' / 'combined_exclusion_30m.tif'
load_zones_path = base_path / "input_files" / "load_zones" / "load_zones.shp"

# Intermediate output paths
ref_footprints_output_path = base_path / 'intermediate_outputs' / 'ref_footprints' 
filtered_nlcd_ag_path = base_path / 'intermediate_outputs' / 'nlcd_ag_filtered'
with_dists_path = base_path / 'intermediate_outputs' / 'with_dists'
infrastructure_overlaps_removed_path = base_path / 'intermediate_outputs' / 'no_infrastructure_overlaps'

# Final output path for suitable candidate sites
final_output_path = base_path.parent / 'candidate_sites'

# Create output directories if they don't exist
for path in [ref_footprints_output_path, filtered_nlcd_ag_path, with_dists_path, infrastructure_overlaps_removed_path, final_output_path]:
    path.mkdir(parents=True, exist_ok=True)

In [9]:
# Define reference plant specifications

# Create a dictionary mapping each hydrogen production technology to its square reference plant footprint length (m)
ref_footprint = {
    "gas_smr": 354.5,
    "gas_smr_ccs": 354.5,
    "bio_smr": 354.5,
    "bio_smr_ccs": 354.5,
    "gas_atr_ccs": 354.5,
    "bio_atr_ccs": 354.5,
    "coal_gas": 614.0,
    "coal_gas_ccs": 614.0,
    "biomass_gas": 297.6,
}

# Create a dictionary mapping each hydrogen production technology to its reference plant capacity (tonnes/day)
ref_capacity = {
    "gas_smr": 150,
    "gas_smr_ccs": 150,
    "bio_smr": 150,
    "bio_smr_ccs": 150,
    "gas_atr_ccs": 205,
    "bio_atr_ccs": 205,
    "coal_gas": 205,
    "coal_gas_ccs": 205,
    "biomass_gas": 48,
}

Step 1: Break down each 1x1km suitable square into smaller squares based on the reference plant footprint lengths

In [10]:
# Define a helper function that a length (in meters) to the nearest length greater than or equal to the original length that evenly divide a 
# 1x1 km grid into squares.

def nearest(length):
    if length <= 250:
        return 250
    elif length <= 1000/3:
        return 1000/3
    elif length <= 500:
        return 500
    elif length <= 1000:
        return 1000

# Use the helper to map the reference footprint length for each tech to the length we'll use to break down the 1x1km squares.
rounded_ref_capacity = {k: nearest(v) for k, v in ref_footprint.items()}

In [11]:
# Run processing
for file_path in candidate_sites_path.glob("*.gpkg"):
    print('Processing:', file_path.name)
    # Extract the technology name from the file name
    tech_name = file_path.stem
    
    # Load the candidate sites GeoPackage
    gdf = gpd.read_file(file_path)
    
    # Get the reference plant footprint length for the current technology
    ref_length = rounded_ref_capacity.get(tech_name)
    
    # Calculate the number of smaller squares along each side of the 1x1 km square
    num_squares_per_side = int(1000 / ref_length)
    
    suitable_sites = []
    
    # Iterate over each geometry in the GeoDataFrame
    for _, row in gdf.iterrows():

        minx, miny, maxx, maxy = row.geometry.bounds
        
        # Generate smaller squares within the 1x1 km square
        for i in range(num_squares_per_side):
            for j in range(num_squares_per_side):
                new_minx = minx + i * ref_length
                new_miny = miny + j * ref_length
                new_maxx = new_minx + ref_length
                new_maxy = new_miny + ref_length
                
                new_square = box(new_minx, new_miny, new_maxx, new_maxy)
                suitable_sites.append(new_square)
    
    # Create a GeoDataFrame from the suitable sites
    suitable_gdf = gpd.GeoDataFrame(geometry=suitable_sites, crs=gdf.crs)
                      
    suitable_gdf.to_file(ref_footprints_output_path / f"{tech_name}.gpkg", driver="GPKG")
    
    print(f"Processed sites with reference footprints for {tech_name}.")


Processing: biomass_gas.gpkg
Processed sites with reference footprints for biomass_gas.
Processing: gas_atr_ccs.gpkg
Processed sites with reference footprints for gas_atr_ccs.
Processing: coal_gas_ccs.gpkg
Processed sites with reference footprints for coal_gas_ccs.
Processing: bio_atr_ccs.gpkg
Processed sites with reference footprints for bio_atr_ccs.
Processing: gas_smr_ccs.gpkg
Processed sites with reference footprints for gas_smr_ccs.
Processing: bio_smr_ccs.gpkg
Processed sites with reference footprints for bio_smr_ccs.
Processing: bio_smr.gpkg
Processed sites with reference footprints for bio_smr.
Processing: coal_gas.gpkg
Processed sites with reference footprints for coal_gas.
Processing: gas_smr.gpkg
Processed sites with reference footprints for gas_smr.


Step 2: Apply the NLCD and Farms Under Threat datasets (each has a 30m resolution) to filter the candidates in the newly obtained layers. 

Using the NLCD, we filter out: Open Water, Pernnial Ice/Snow, Developed Open Space, Developed Low Intensity, Developed Medium Intensity, Developed High Intensity, Deciduous Forest, Evergreen Forest, Mixed Forest, Woody Wetlands, and Herbaceous Wetlands.

Using the Farms under Threat dataset, we filter out "Nationally Significant Agricultural Land."

For ease of processing, we combined the two datasets into one raster file. Excluded pixels have a value of 1.

Note: If 95% or more of a candidate site passes the filter, it is deemed acceptable and kept.

In [12]:
# Create a helper function to calculate the acceptance of a candidate site using an exclusion raster
def calculate_acceptance(geometry, exclusion_src, threshold=0.95):
    # Get bounding box
    minx, miny, maxx, maxy = geometry.bounds
    
    # Compute window for each raster
    exclusion_window = from_bounds(minx, miny, maxx, maxy, exclusion_src.transform)
    
    # Read only windowed data
    exclusion_data = exclusion_src.read(1, window=exclusion_window)
    
    window_transform = exclusion_src.window_transform(exclusion_window)
    mask = geometry_mask(
        [geometry],
        transform=window_transform,
        invert=True,
        out_shape=exclusion_data.shape,
        all_touched=True
    )

    exclusion_window_values = exclusion_data[mask]
    
    valid_ratio = np.count_nonzero(exclusion_window_values == 0) / exclusion_window_values.size 
    return valid_ratio >= threshold

In [13]:
# Read in the exclusion raster
exclusion_src = rasterio.open(combined_exclusion_30m_path)

# Process each layer, using the intermediate files created in the previous step
for file_path in ref_footprints_output_path.glob("*.gpkg"):
    print('Processing:', file_path.name)
    tech_name = file_path.stem
    
    refined_gdf = gpd.read_file(file_path)
    
    accepted_geometries = []
    for _, row in refined_gdf.iterrows():
        if calculate_acceptance(row.geometry, exclusion_src):
            accepted_geometries.append(row.geometry)
    
    final_gdf = gpd.GeoDataFrame(geometry=accepted_geometries, crs=refined_gdf.crs)
    final_gdf.to_file(filtered_nlcd_ag_path / f"{tech_name}.gpkg", driver="GPKG")
    print(f"Saved filtered suitable sites for {tech_name}")

Processing: biomass_gas.gpkg
Saved filtered suitable sites for biomass_gas
Processing: gas_atr_ccs.gpkg


KeyboardInterrupt: 

Step 3: Add a load area column to each file, filtering out candidates that do not fall squarely within a load zone

In [ ]:
# Read in the load zones file
load_zones_gdf = gpd.read_file(load_zones_path)

# Process each layer
for file_path in filtered_nlcd_ag_path.glob("*.gpkg"):
    print('Processing:', file_path.name)
    tech_name = file_path.stem
    
    final_gdf = gpd.read_file(file_path)
    
    # Perform spatial join to associate each candidate site with a load zone
    joined_gdf = gpd.sjoin(final_gdf, load_zones_gdf[['geometry', 'LOAD_AREA']], how="left", predicate="within").reset_index(drop=True)
    
    # Filter out geometries that do not fall within any load zone
    joined_gdf = joined_gdf[~joined_gdf["LOAD_AREA"].isna()].copy().drop(columns=["index_right"])

    # Save the updated gdf back to the same file
    joined_gdf.to_file(file_path, driver="GPKG")
    print(f"Added load area info and saved for {tech_name}")

Step 4: Use PostgreSQL to add distances to feedstock sources and substations. Use the scripts in the 'postgres' folder. Put the resulting files in the folder 'candidate_sites_with_dists'

Step 5: Remove overlaps with existing physical features (substations, coal mines, biomass plants, etc).

In [15]:
for tech_file in with_dists_path.glob("*.gpkg"):
    tech_name = tech_file.stem
    gdf = gpd.read_file(tech_file)

    # Remove any candidate sites that overlap with substations
    gdf = gdf[gdf["dist_to_substation_meters"] > 0].copy()

    # Remove any candidates that overlap with feedstock sources (except for natural gas, which may have pipelines underground)
    if tech_name not in ['gas_smr', 'gas_smr_ccs', 'gas_atr_ccs']:
        gdf = gdf[gdf["dist_to_feedstock_meters"] > 0].copy()

    gdf.to_file(infrastructure_overlaps_removed_path / f"{tech_name}.gpkg", driver="GPKG")

Step 6: Remove overlaps between the different technology layers.

In [16]:
# Make a helper function to remove overlaps between layers
def remove_overlaps(base_gdfs, top_gdf):
    """
    Removes overlapping features from the top GeoDataFrame 
    where they overlap with the base GeoDataFrame.
    
    Parameters
    - base_gdf : A list of base layers (overlaps from top will be removed here).
    - top_gdf : The top layer (features overlapping base will be removed).
    
    Returns
    A GeoDataFrame consisting of:
    - only the non-overlapping portions of features from top_gdf
    """
    # Start with first base gdf, geometry only
    combined_base = base_gdfs[0][["geometry"]]

    # Overlay the rest, geometry only
    for base_gdf in base_gdfs[1:]:
        combined_base = gpd.overlay(combined_base, base_gdf[["geometry"]], how="union")

    # Spatial join to find overlapping top features
    overlaps = gpd.sjoin(top_gdf, combined_base, how="inner", predicate="intersects")

    # Keep only those NOT in overlaps
    cleaned_top = top_gdf.loc[~top_gdf.index.isin(overlaps.index)]
    
    return cleaned_top

In [17]:
# Read in the gdfs in order of increasing number of candidate sites
biomass_gdf = gpd.read_file(infrastructure_overlaps_removed_path / 'biomass_gas.gpkg')
bio_smr_gdf = gpd.read_file(infrastructure_overlaps_removed_path / 'bio_smr.gpkg')
bio_smr_ccs_gdf = gpd.read_file(infrastructure_overlaps_removed_path / 'bio_smr_ccs.gpkg')
coal_gas_ccs_gdf = gpd.read_file(infrastructure_overlaps_removed_path / 'coal_gas_ccs.gpkg')
coal_gas_gdf = gpd.read_file(infrastructure_overlaps_removed_path / 'coal_gas.gpkg')
gas_smr_gdf = gpd.read_file(infrastructure_overlaps_removed_path / 'gas_smr.gpkg')

In [ ]:
# Remove overlaps in order of increasing number of candidate sites

# The filtered sites for biomass are the same as the original
biomass_gdf.to_file(final_output_path / 'biomass_gas.gpkg', driver='GPKG')

# Get the combined sites for biomass smr and biomass smr + ccs
all_bio_smr_gdf = remove_overlaps([biomass_gdf], bio_smr_ccs_gdf)
all_bio_smr_gdf.to_file(final_output_path / 'all_bio_smr_atr.gpkg', driver='GPKG')

# Get the sites for biomass smr + ccs only
bio_smr_only_gdf = remove_overlaps([biomass_gdf, all_bio_smr_gdf], bio_smr_gdf)
bio_smr_only_gdf.to_file(final_output_path / 'extra_bio_smr.gpkg', driver='GPKG')

# Get the combined sites for coal gasification and coal gasification + ccs
all_coal_gas_gdf = remove_overlaps([biomass_gdf, all_bio_smr_gdf, bio_smr_only_gdf], coal_gas_ccs_gdf)
all_coal_gas_gdf.to_file(final_output_path / 'all_coal_gas.gpkg', driver='GPKG')

# Get the sites for coal gasification + ccs only
coal_gas_only_gdf = remove_overlaps([biomass_gdf, all_bio_smr_gdf, bio_smr_only_gdf, all_coal_gas_gdf], coal_gas_gdf)
coal_gas_only_gdf.to_file(final_output_path / 'extra_coal_gas.gpkg', driver='GPKG')

# Get the sites for all gas smr
all_gas_smr_gdf = remove_overlaps([biomass_gdf, all_bio_smr_gdf, bio_smr_only_gdf, all_coal_gas_gdf, coal_gas_only_gdf], gas_smr_gdf)
all_gas_smr_gdf.to_file(final_output_path / 'all_gas_smr_atr.gpkg', driver='GPKG')

Step 7: Calculate the potential of each technology in each load zone

In [22]:
# Create helper function that calculates the potential capacity per load zone for given tech(s)
def calculate_potential(candidates_gdf, tech_names, ref_capacity):
    """
    Inputs: 
    - candidates_gdf: the gdf of candidate sites for the given tech(s)
    - tech_names: the hydrogen production technologies that the layer is for
    - ref_capacity: the reference capacity of the candidates (tonnes/day)
        - each technology in tech_names must have the same reference capacity

    Outputs:
    - df: a df with the potential capacity per tech by load zone, structured with the following columns:
        - LOAD_AREA, prod_tech1, prod_tech2, prod_tech3, site_count, potential_MW
        - prod_tech2 and prod_tech3 may be empty (contain empty strings)
    """

    # Count the number of candidate sites in each load zone
    count_by_load_area = candidates_gdf.groupby("LOAD_AREA").size().reset_index(name="site_count")

    for i in range(1, 4):  
        if i <= len(tech_names):
            count_by_load_area[f"prod_tech{i}"] = tech_names[i-1]
        else:
            count_by_load_area[f"prod_tech{i}"] = ""


    # Calculate reference capacity in MW (using 33.39 kg H2/MWh and truncating)
    tech_ref_capacity_MW = int(ref_capacity / 24 * 33.39)

    # Calculate total potential capacity in each load zone
    count_by_load_area["potential_MW"] = (
        count_by_load_area["site_count"] * tech_ref_capacity_MW
    )

    return count_by_load_area


In [ ]:
# Create a running list of potential capacity
output_df = pd.DataFrame()

# Load final suitable candidate sites for each technology and calculate potential capacity by load zone
for tech_file in final_output_path.glob("*.gpkg"):
    gdf = gpd.read_file(tech_file)

    file_name = tech_file.stem
    
    if file_name == 'all_bio_smr_atr':
        tech_names = ['bio_smr', 'bio_smr_ccs', 'bio_atr_ccs']
    elif file_name == 'all_coal_gas':
        tech_names = ['coal_gas', 'coal_gas_ccs']
    elif file_name == 'all_gas_smr_atr':
        tech_names = ['gas_smr', 'gas_smr_ccs', 'gas_atr_ccs']
    elif file_name == 'biomass_gas':
        tech_names = ['biomass_gas']
    elif file_name == 'extra_bio_smr':
        tech_names = ['bio_smr']
    elif file_name == 'extra_coal_gas':
        tech_names = ['coal_gas']
    else:
        raise Exception(f'tech name {file_name} not found')

    nameplate_capacity = ref_capacity[tech_names[0]]

    potential_df = calculate_potential(gdf, tech_names, nameplate_capacity)
    
    # Append to output DataFrame
    output_df = pd.concat([output_df, potential_df], ignore_index=True)


# Sort 
output_df = output_df.sort_values(by=["LOAD_AREA", "prod_tech1", "prod_tech2", "prod_tech3"])

# Build the cartesian product of load areas × unique technology sets
load_areas = load_zones_gdf["LOAD_AREA"].unique()

# Get unique tech sets (as tuples) from your output_df
tech_sets = (
    output_df[["prod_tech1", "prod_tech2", "prod_tech3"]]
    .drop_duplicates()
    .apply(tuple, axis=1)
    .tolist()
)

# Build MultiIndex product of load_areas × tech_sets
all_combinations = pd.MultiIndex.from_product(
    [load_areas, tech_sets],
    names=["LOAD_AREA", "tech_set"]
).to_frame(index=False)

# Expand the tuple back into columns
all_combinations[["prod_tech1", "prod_tech2", "prod_tech3"]] = pd.DataFrame(
    all_combinations["tech_set"].tolist(), index=all_combinations.index
)
all_combinations = all_combinations.drop(columns="tech_set")

# Merge with output_df on LOAD_AREA + all three prod_tech columns
output_df = all_combinations.merge(
    output_df,
    on=["LOAD_AREA", "prod_tech1", "prod_tech2", "prod_tech3"],
    how="left"
).fillna(0)

# Save
output_csv_path = base_path.parent / "technology_capacity_by_load_zone.csv"
output_df.to_csv(output_csv_path, index=False)
print(f"Saved technology capacity by load zone to {output_csv_path}")


Saved technology capacity by load zone to /Users/nicholaskong/Desktop/REAM_lab/hydrogen_siting/technology_capacity_by_load_zone.csv
